In [114]:
#Package Imports
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import csv
import sklearn.metrics as metrics
from subprocess import check_output
import seaborn as sns
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)
from pyspark.sql.functions import col, countDistinct
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlContext = SQLContext(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local) created by __init__ at <ipython-input-6-23ae3bfd1d13>:13 

In [7]:
!conda install -y -c conda-forge xgboost

Solving environment: done

## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/python3

  added / updated specs: 
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxgboost-0.80            |       hfc679d8_1         3.4 MB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    py-xgboost-0.80            |   py36hfc679d8_1          61 KB  conda-forge
    xgboost-0.80               |   py36hfc679d8_1           8 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.4 MB

The following NEW packages will be INSTALLED:

    _py-xgboost-mutex: 2.0-cpu_0            conda-forge
    libxgboost:        0.80-hfc679d8_1      conda-forge
    py-xgboost:        0.80-py36hfc679d8_1  conda-forge
    xgboost:           0.80-py36hfc679d8_1  co

In [81]:
#Create Data Schema
schema_T = StructType([StructField("resident_status", StringType(), True)\
                    ,StructField("education_eithy_nine", IntegerType(), True)\
                    ,StructField("education_o_three", IntegerType(), True)\
                    ,StructField("education_flag", IntegerType(), True)\
                    ,StructField("month_of_death", IntegerType(), True)\
                    ,StructField("sex", StringType(), True)\
                    ,StructField("detail_age_type", IntegerType(), True)\
                    ,StructField("detail_age", IntegerType(), True)\
                    ,StructField("age_substitution_flag", IntegerType(), True)\
                    ,StructField("age_recode_fifty_two", IntegerType(), True)\
                    ,StructField("age_recode_twenty_seven", IntegerType(), True)\
                    ,StructField("age_recode_twelve", IntegerType(), True)\
                    ,StructField("infant_age_recode_twenty_two", IntegerType(), True)\
                    ,StructField("place_of_death_and_decedents_status", IntegerType(), True)\
                    ,StructField("marital_status", StringType(), True)\
                    ,StructField("day_of_week_of_death", IntegerType(), True)\
                    ,StructField("current_data_year",  IntegerType(), True)\
                    ,StructField("injury_at_work", StringType(), True)\
                    ,StructField("manner_of_death", IntegerType(), True)\
                    ,StructField("method_of_disposition", StringType(), True)\
                    ,StructField("autopsy", StringType(), True)\
                    ,StructField("activity_code", IntegerType(), True)\
                    ,StructField("place_of_injury_for_causes", IntegerType(), True)\
                    ,StructField("icd_code_tenth_revision", StringType(), True)\
                    ,StructField("three_fifty_eight_cause_recode", IntegerType(), True)\
                    ,StructField("one_thirteen_cause_recode ", IntegerType(), True)\
                    ,StructField("one_thirty_infant_cause_recode", IntegerType(), True)\
                    ,StructField("thirty_nine_cause_recode ", IntegerType(), True)\
                    ,StructField("number_of_entity_axis_conditions ", IntegerType(), True)\
                    ,StructField("entity_condition_one", StringType(), True)\
                    ,StructField("entity_condition_two", StringType(), True)\
                    ,StructField("entity_condition_three", StringType(), True)\
                    ,StructField("entity_condition_four", StringType(), True)\
                    ,StructField("entity_condition_five", StringType(), True)\
                    ,StructField("entity_condition_six", StringType(), True)\
                    ,StructField("entity_condition_seven", StringType(), True)\
                    ,StructField("entity_condition_eight", StringType(), True)\
                    ,StructField("entity_condition_nine", StringType(), True)\
                    ,StructField("entity_condition_ten", StringType(), True)\
                    ,StructField("entity_condition_eleven", StringType(), True)\
                    ,StructField("entity_condition_twelve", StringType(), True)\
                    ,StructField("entity_condition_thirteen", StringType(), True)\
                    ,StructField("entity_condition_fourteen", StringType(), True)\
                    ,StructField("entity_condition_fifteen", StringType(), True)\
                    ,StructField("entity_condition_sixteen", StringType(), True)\
                    ,StructField("entity_condition_seventeen", StringType(), True)\
                    ,StructField("entity_condition_eighteen", StringType(), True)\
                    ,StructField("entity_condition_nineteen", StringType(), True)\
                    ,StructField("entity_condition_twenty", StringType(), True)\
                    ,StructField("number_of_record_axis_conditions", IntegerType(), True)\
                    ,StructField("record_condition_one", StringType(), True)\
                    ,StructField("record_condition_two", StringType(), True)\
                    ,StructField("record_condition_three", StringType(), True)\
                    ,StructField("record_condition_four", StringType(), True)\
                    ,StructField("record_condition_five", StringType(), True)\
                    ,StructField("record_condition_six", StringType(), True)\
                    ,StructField("record_condition_seven", StringType(), True)\
                    ,StructField("record_condition_eight", StringType(), True)\
                    ,StructField("record_condition_nine", StringType(), True)\
                    ,StructField("record_condition_ten", StringType(), True)\
                    ,StructField("record_condition_eleven", StringType(), True)\
                    ,StructField("record_condition_twelve", StringType(), True)\
                    ,StructField("record_condition_thirteen", StringType(), True)\
                    ,StructField("record_condition_fourteen", StringType(), True)\
                    ,StructField("record_condition_fifteen", StringType(), True)\
                    ,StructField("record_condition_sixteen", StringType(), True)\
                    ,StructField("record_condition_seventeen", StringType(), True)\
                    ,StructField("record_condition_eighteen", StringType(), True)\
                    ,StructField("record_condition_nineteen", StringType(), True)\
                    ,StructField("record_condition_twenty", StringType(), True)\
                    ,StructField("race", IntegerType(), True)\
                    ,StructField("bridged_race_flag", IntegerType(), True)\
                    ,StructField("race_imputation_flag", IntegerType(), True)\
                    ,StructField("race_recode_three", IntegerType(), True)\
                    ,StructField("race_recode_five", IntegerType(), True)\
                    ,StructField("hispanic_origin", IntegerType(), True)\
                    ,StructField("hispanic_originrace_recode", IntegerType(), True)\
                    ])
schema_ICD = StructType([StructField("code", StringType(), True)\
                    ,StructField("description", StringType(), True)\
                    ])

In [101]:
#Load ICD Description Dataset
df_icd = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('Section111ValidICD10-2018.csv', schema=schema_ICD)
df_icd.count()

70938

In [105]:
df_icd.dtypes

[('code', 'string'), ('description', 'string')]

In [104]:
#Load the Merged data
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('all_mortality.csv', schema=schema_T)
df.count()

27720683

In [100]:
df.dtypes

[('resident_status', 'string'),
 ('education_eithy_nine', 'int'),
 ('education_o_three', 'int'),
 ('education_flag', 'int'),
 ('month_of_death', 'int'),
 ('sex', 'string'),
 ('detail_age_type', 'int'),
 ('detail_age', 'int'),
 ('age_substitution_flag', 'int'),
 ('age_recode_fifty_two', 'int'),
 ('age_recode_twenty_seven', 'int'),
 ('age_recode_twelve', 'int'),
 ('infant_age_recode_twenty_two', 'int'),
 ('place_of_death_and_decedents_status', 'int'),
 ('marital_status', 'string'),
 ('day_of_week_of_death', 'int'),
 ('current_data_year', 'int'),
 ('injury_at_work', 'string'),
 ('manner_of_death', 'int'),
 ('method_of_disposition', 'string'),
 ('autopsy', 'string'),
 ('activity_code', 'int'),
 ('place_of_injury_for_causes', 'int'),
 ('icd_code_tenth_revision', 'string'),
 ('three_fifty_eight_cause_recode', 'int'),
 ('one_thirteen_cause_recode ', 'int'),
 ('one_thirty_infant_cause_recode', 'int'),
 ('thirty_nine_cause_recode ', 'int'),
 ('number_of_entity_axis_conditions ', 'int'),
 ('enti

# EXPLORATORY DATA ANALYSES

# Business Question 1: What happens to Americans after Death?

In [11]:
#VERSION 1
print('Corpse Dispositions Stats by Year')
df.groupBy(df[16], df[19]).count().sort(df[16]).show(100)

Corpse Dispositions Stats by Year
+-----------------+---------------------+-------+
|current_data_year|method_of_disposition|  count|
+-----------------+---------------------+-------+
|                0|                    m|     10|
|             2005|                    U|1489091|
|             2005|                    E|  21247|
|             2005|                    R|  31954|
|             2005|                    C| 350018|
|             2005|                    D|   4795|
|             2005|                    B| 553202|
|             2005|                    O|   2199|
|             2006|                    O|   2252|
|             2006|                    U|1266857|
|             2006|                    C| 423282|
|             2006|                    E|  23412|
|             2006|                    D|   6883|
|             2006|                    B| 667169|
|             2006|                    R|  40870|
|             2007|                    E|  26139|
|             20

In [ ]:
#VERSION 2
print('Corpse Dispositions Stats by Year')
df.registerTempTable("data")
methods = sqlContext.sql("\
    SELECT current_data_year AS Year, \
        CASE method_of_disposition \
            WHEN 'C' THEN 'Cremation' \
            WHEN 'B' THEN 'Burial' \
            WHEN 'D' THEN 'Donation' \
            WHEN 'E' THEN 'E' \
            WHEN 'O' THEN 'Other' \
            WHEN 'R' THEN 'RemovedFromUSA' \
            WHEN 'U' THEN 'Unknown' \
            END AS MethodofDisposition, \
        COUNT(*) AS Count \
        FROM data \
        GROUP BY 1, 2\
        ORDER BY 1, 3\
    ").show(1000)

# Business Question 2: Leading Cuases and Manner of Death by Year

In [13]:
#VERSION 1
print('Leading Manner of Deaths in the USA by Year')
mannerOfDeath = df.groupBy(df[16], df[18]).count().sort(df[16]).show(200)

Leading Manner of Deaths in the USA by Year
+-----------------+---------------+-------+
|current_data_year|manner_of_death|  count|
+-----------------+---------------+-------+
|                0|              0|     10|
|             2005|              5|   9131|
|             2005|              1| 113949|
|             2005|              3|  19235|
|             2005|              0| 732891|
|             2005|              2|  32934|
|             2005|              7|1537327|
|             2005|              4|   7039|
|             2006|              3|  19773|
|             2006|              1| 118224|
|             2006|              7|1549667|
|             2006|              4|   4651|
|             2006|              0| 693733|
|             2006|              2|  33562|
|             2006|              5|  11115|
|             2007|              0| 680409|
|             2007|              1| 119657|
|             2007|              5|  10886|
|             2007|             

In [60]:
#VERSION 2
print('Leading Manner of Deaths in the USA by Year')
sqlContext.sql("\
    SELECT current_data_year AS Year, \
        CASE manner_of_death \
            WHEN 0 THEN 'Not Specified' \
            WHEN '1' THEN 'Accident' \
            WHEN '2' THEN 'Suicide' \
            WHEN '3' THEN 'Homicide' \
            WHEN '4' THEN 'Pending investigation' \
            WHEN '5' THEN 'Could not be determine' \
            WHEN '6' THEN 'Self-Inflicted' \
            WHEN '7' THEN 'Natural' \
            END AS MannerOfDeath, \
        COUNT(*) AS Count \
        FROM data \
        GROUP BY 1, 2\
        ORDER BY 1, 3\
    ").show(1000)

+----+--------------------+-------+
|Year|       MannerOfDeath|  Count|
+----+--------------------+-------+
|   0|       Not Specified|     10|
|2005|Pending investiga...|   7039|
|2005|Could not be dete...|   9131|
|2005|            Homicide|  19235|
|2005|             Suicide|  32934|
|2005|            Accident| 113949|
|2005|       Not Specified| 732891|
|2005|             Natural|1537327|
|2006|Pending investiga...|   4651|
|2006|Could not be dete...|  11115|
|2006|            Homicide|  19773|
|2006|             Suicide|  33562|
|2006|            Accident| 118224|
|2006|       Not Specified| 693733|
|2006|             Natural|1549667|
|2007|Pending investiga...|   4251|
|2007|Could not be dete...|  10886|
|2007|            Homicide|  19501|
|2007|             Suicide|  34827|
|2007|            Accident| 119657|
|2007|       Not Specified| 680409|
|2007|             Natural|1558812|
|2008|Pending investiga...|   7419|
|2008|Could not be dete...|  10759|
|2008|            Homicide| 

In [68]:
print('Leading 20 Causes of Deaths in the USA by Year')
df_icd.registerTempTable("data_icd")
manner = sqlContext.sql("\
    SELECT DISTINCT data_icd.description, COUNT(data.icd_code_tenth_revision) \
        FROM data_icd JOIN data \
        ON data.icd_code_tenth_revision = data_icd.code \
        GROUP BY data_icd.description \
        ORDER BY 2 DESC \
        LIMIT 20 \
    ").show()

Leading Causes of Deaths in the USA by Year
+--------------------+------------------------------+
|         description|count(icd_code_tenth_revision)|
+--------------------+------------------------------+
|Acute myocardial ...|                       1388327|
|Chronic obstructi...|                       1136896|
|Alzheimer's disea...|                        896200|
|Pneumonia, unspec...|                        512409|
|Malignant neoplas...|                        451882|
|Malignant neoplas...|                        403164|
|Sepsis, unspecifi...|                        368875|
|Malignant neoplas...|                        311877|
|Hypertensive hear...|                        251086|
| Parkinson's disease|                        246814|
|Pneumonitis due t...|                        194079|
|Cardiomyopathy, u...|                        192955|
|Essential (primar...|                        190549|
|Chronic obstructi...|                        177701|
|Malignant neoplas...|                

# Predictive Model: 

In [75]:
from pyspark.mllib.clustering import KMeans

filename = 'all_mortality.csv'
data = sc.textFile(filename)
K = 5

[array([2.00750488e+03, 1.23996237e+00]), array([2.00549772e+03, 1.23480844e+00]), array([2010.35401701,   63.64384885]), array([2.01001060e+03, 1.25157888e+00]), array([2.01352563e+03, 1.27847561e+00])]


In [76]:
#Does Race Determine Manner of Death?
data.map(lambda line: line.split(","))\
   .map(lambda fields: [float(fields[16]),float(fields[70])])\
   .filter(lambda point: sum(point) != 0)\
   .persist()

clusters = KMeans.train(data, K, maxIterations=100, runs=1, initializationMode="random")
center = clusters.clusterCenters
print (center)

[array([2.01352563e+03, 1.27847561e+00]), array([2.00750360e+03, 1.31923546e+00]), array([2.00549734e+03, 1.18828149e+00]), array([2010.35401701,   63.64384885]), array([2.01001165e+03, 1.22957572e+00])]


In [106]:
#Does Age Determine Method of Disposition?
data.map(lambda line: line.split(","))\
   .map(lambda fields: [float(fields[7]),float(fields[19])])\
   .filter(lambda point: sum(point) != 0)\
   .persist()

clusters = KMeans.train(data, K, maxIterations=100, runs=1, initializationMode="random")
center = clusters.clusterCenters
print (center)



[array([2.00650327e+03, 1.23739046e+00]), array([2010.35401701,   63.64384885]), array([2.01051836e+03, 1.25519391e+00]), array([2.01401468e+03, 1.28255731e+00])]


NameError: name 'LogisticRegression' is not defined

In [109]:
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(df)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

NameError: name 'LogisticRegression' is not defined

In [111]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(training)
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(df)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

NameError: name 'training' is not defined

In [112]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(df)
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(df)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

NameError: name 'training' is not defined

In [113]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(df)
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(df)

# Print the coefficients and intercepts for logistic regression with multinomial family
print("Multinomial coefficients: " + str(mlrModel.coefficientMatrix))
print("Multinomial intercepts: " + str(mlrModel.interceptVector))

IllegalArgumentException: 'Field "features" does not exist.'

In [115]:
#Load the Merged data
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load('all_mortality.csv', schema=schema_T)
df.count()

27720683

In [116]:
df.count()

27720683

In [ ]:
df.count()


In [126]:
new_df = df.toDF()

IllegalArgumentException: "requirement failed: The number of columns doesn't match.\nOld column names (77): resident_status, education_eithy_nine, education_o_three, education_flag, month_of_death, sex, detail_age_type, detail_age, age_substitution_flag, age_recode_fifty_two, age_recode_twenty_seven, age_recode_twelve, infant_age_recode_twenty_two, place_of_death_and_decedents_status, marital_status, day_of_week_of_death, current_data_year, injury_at_work, manner_of_death, method_of_disposition, autopsy, activity_code, place_of_injury_for_causes, icd_code_tenth_revision, three_fifty_eight_cause_recode, one_thirteen_cause_recode , one_thirty_infant_cause_recode, thirty_nine_cause_recode , number_of_entity_axis_conditions , entity_condition_one, entity_condition_two, entity_condition_three, entity_condition_four, entity_condition_five, entity_condition_six, entity_condition_seven, entity_condition_eight, entity_condition_nine, entity_condition_ten, entity_condition_eleven, entity_conditi

In [125]:
new_df.printSchema()

AttributeError: 'function' object has no attribute 'printSchema'

In [118]:
df.show()


+---------------+--------------------+-----------------+--------------+--------------+---+---------------+----------+---------------------+--------------------+-----------------------+-----------------+----------------------------+-----------------------------------+--------------+--------------------+-----------------+--------------+---------------+---------------------+-------+-------------+--------------------------+-----------------------+------------------------------+--------------------------+------------------------------+-------------------------+---------------------------------+--------------------+--------------------+----------------------+---------------------+---------------------+--------------------+----------------------+----------------------+---------------------+--------------------+-----------------------+-----------------------+-------------------------+-------------------------+------------------------+------------------------+--------------------------+--------